This notebook demonstrates the implementation and application of a multivariate linear regression model for the "housing" dataset. You can download the dataset from http://lib.stat.cmu.edu/datasets/.

**Comments:** "*The file cadata.txt contains all the the variables. Specifically, it contains median house value, median income, housing median age, total rooms, total bedrooms, population, households, latitude, and longitude in that order. The dependent variable is ln(median house value).*"

In [ ]:
import numpy

# load data from text file (note that the initial comments have been deleted from the original file)
data = numpy.loadtxt("cadata.txt")

# the first column corresponds to the target variables; the remaining ones are the features
y, X = data[:,0], data[:,1:]

Next, we define a class for our linear regression model that implements the "fitting" phase (computation of the weights) and the "prediction" phase (computation of predictions for new data points).

In [ ]:
class LinearRegression():
    """
    Linear regression implementation.
    """

    def __init__(self, algorithm="solve", lr=0.01, maxiter=1000):
        """
        Instantiates the model object.

        Parameters
        ----------
        algorithm : str, default="solve"
            The type of algorithm that shall be applied.
        lr : float, default 0.02
            The learning rate for gradient descent.
        maxiter : int, default 100
            The maximal number of iterations for gradient descent
        """           
        
        self.algorithm = algorithm
        self.lr = lr
        self.maxiter = maxiter
        
    def fit(self, X, y):
        """
        Fits the linear regression model.

        Parameters
        ----------
        X : Array of shape [n_samples, n_features]
        y : Array of shape [n_samples, 1]
        """        

        # make sure that we have multidimensional numpy arrays
        X = numpy.array(X).reshape((X.shape[0], -1))
        # IMPORTANT: Make sure that we have a column vector! 
        y = numpy.array(y).reshape((len(y), 1))

        # prepend a column of ones
        ones = numpy.ones((X.shape[0], 1))
        X = numpy.concatenate((ones, X), axis=1)           

        # compute weights
        if self.algorithm == "solve":
            
            XtX_pinv = numpy.linalg.pinv(numpy.dot(X.T, X))
            Xty = numpy.dot(X.T,y)
            self._w = numpy.dot(XtX_pinv, Xty)
            
        elif self.algorithm == "gradient":
            
            # initialize with zeros (alternatively: small random values)
            self._w = numpy.zeros((X.shape[1], 1))
            
            # TODO: YOUR CODE HERE
                            
        else:
            
            raise Exception("Algorithm {} not implemented!".format(self.algorithm))
                
    def predict(self, X, add_ones=True):
        """
        Computes predictions for a new set of points.

        Parameters
        ----------
        X : Array of shape [n_samples, n_features]

        Returns
        -------
        predictions : Array of shape [n_samples, 1]
        """                     

        # make sure that we have multidimensional numpy arrays
        X = numpy.array(X).reshape((X.shape[0], -1))

        if add_ones == True:
            # prepend a column of ones
            ones = numpy.ones((X.shape[0], 1))
            X = numpy.concatenate((ones, X), axis=1)           

        # compute predictions
        predictions = numpy.dot(X, self._w)

        return predictions

Let us normalize the data first. This makes it easier for gradient descent to find the optimal solution.

In [ ]:
from sklearn import preprocessing

scaler = preprocessing.MinMaxScaler()
X = scaler.fit_transform(X)

We first instantiate the "model" object. Afterwards, we call the "fit" method to fit our model (i.e., to compute the weights).

In [ ]:
model = LinearRegression(algorithm="gradient", lr=0.1, maxiter=1000)
model.fit(X, y)

Given the fitted model, we can now obtain predictions for new data points. For simplification, we just use our data points again here.

In [ ]:
preds = model.predict(X)

Finally, we have a look at the quality of our model by computing the RMSE and by generating a plot "predictions" vs. "true labels".

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter

from sklearn.metrics import mean_squared_error

# compute RMSE
print("RMSE: {}".format(numpy.sqrt(mean_squared_error(y, preds))))

# visualize predictions vs. true labels
fig = plt.figure(figsize=(8,8))
plt.scatter(preds, y, color="blue", alpha=0.5)
plt.xticks(rotation=45)
plt.gca().xaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))
plt.plot([-100000,600000], [-100000, 600000], 'k--')
plt.xlabel("Predictions")
plt.ylabel("True Labels")
plt.xlim([-100000,600000])
plt.ylim([-100000,600000])
plt.title("Evaluation of Regression Model")
plt.show()